# LMIR

In [6]:
#!pip install ir-measures

import pandas as pd
import nltk
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import ir_measures
from ir_measures import nDCG, P, Judged
#import nltk
#nltk.download('punkt')
#nltk.download('punkt_tab')  
#import ir_datasets

from funcs import load_datasets, get_docs



In [2]:
datasets = load_datasets(["ru", "zh", "fa"])

documents = pd.DataFrame(datasets["ru"].docs_iter())
# Load the Qrels and Queries
qrels = pd.DataFrame(datasets["ru"].qrels_iter()) 
queries = pd.DataFrame(datasets["ru"].queries_iter()) 


In [9]:
from googletrans import Translator
import pandas as pd
from ir_datasets import load

# Initialize the translator
translator = Translator()


# Translate queries to Russian
translated_queries = []
for _, query in filtered_queries.iterrows():
    translated_title = translator.translate(query["title"], src="en", dest="ru").text if "title" in query else None
    translated_description = translator.translate(query["description"], src="en", dest="ru").text if "description" in query else None

    translated_queries.append({
        "query_id": query["query_id"],
        "original_title": query["title"],
        "translated_title": translated_title,
        "original_description": query["description"],
        "translated_description": translated_description
    })

translated_queries_df = pd.DataFrame(translated_queries)

# Save Translated Queries for Inspection
translated_queries_df.to_excel("translated_queries.xlsx", index=False)


# Save to Excel for inspection
output_file = "translated_queries.xlsx"
translated_queries_df.to_excel(output_file, index=False)

# Display a sample of the translated queries
print("Translated Queries (Sample):")
print(translated_queries_df.head())

# Additional: Check the structure of the queries for inspection
print("\nSample Queries from Dataset:")
print(queries.head())

Translated Queries (Sample):
  query_id                                original_title  \
0        3                    British royal news impacts   
1        6          Gibraltar's Sovereignty After Brexit   
2       13                US-South Korea Trade Agreement   
3       14  North Korean Earthquakes and Nuclear Testing   
4      101      Shipwrecks and Historical European Trade   

                                    translated_title  \
0                    British Royal News воздействует   
1                Суверенитет Гибралтара после Brexit   
2            Торговое соглашение о США и Южном Корее   
3  Землетрясения и ядерные испытания в Северной К...   
4  Кораблекрушения и историческая европейская тор...   

                                original_description  \
0  What political and economic impacts does news ...   
1  How will Gibraltar's sovereignty be impacted b...   
2  How will South Korea benefit from or be harmed...   
3  Are earthquakes in North Korea being caused by

In [3]:
common_query_ids = set(qrels["query_id"]).intersection(queries["query_id"])
filtered_queries = queries[queries["query_id"].isin(common_query_ids)]

In [4]:
documents.head

<bound method NDFrame.head of                                       doc_id  \
0       ecd810c8-4b67-4a53-a0bb-20e0214becde   
1       bdcf1b07-7d19-41a8-923d-55d08957a8d6   
2       b148f67a-8605-48d9-b032-f32a2280f1f0   
3       fcd39864-6cf5-4193-8903-9a101b6863ba   
4       2a0acf64-5fd4-43af-acbf-3f728d65ca2a   
...                                      ...   
964714  ec1ab8fa-faa5-4d34-a150-873a482725c8   
964715  eec74ba1-0566-4122-9660-6a5ca9a94564   
964716  22750309-0e4c-4fe6-9c86-c705008bdeb8   
964717  8804aa1b-3a89-4811-8503-7e3b400bb411   
964718  c89674e1-fbaf-48f9-9ce2-2517968dc020   

                                                    title  \
0       Рафаэль Надаль – в четвертьфинале Открытого че...   
1       Житель Октябрьского района, обналичив чужую ка...   
2       Воспоминания участников войны в Афганистане из...   
3         Глава спецслужбы ФРГ Масен отправлен в отставку   
4       Европейские индексы - 02-04-18 | Новости Армен...   
...                        

In [5]:
# Preprocessing: tokenize and normalize

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def preprocess(text):
    # Tokenize
    tokens = word_tokenize(text)
    # Lowercase and remove punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)  # Join tokens back into a single string

In [6]:
# Preprocess: tokenize and normalize - Russian

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


def preprocess_ru(text):
    # Tokenize
    tokens = word_tokenize(text)
    # Lowercase and remove punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)  # Join tokens back into a single string

In [7]:
#!pip install pymystem3
# Preprocess: using Mystem

from pymystem3 import Mystem
from nltk.corpus import stopwords

# Initialize Mystem and stopwords
mystem = Mystem()
stop_words = set(stopwords.words('russian'))

def preprocess_ru(text):
    """
    Preprocess Russian text using Mystem.
    - Tokenize and lemmatize
    - Lowercase
    - Remove punctuation and stopwords
    """
    # Tokenize and lemmatize using Mystem
    tokens = mystem.lemmatize(text)
    # Filter tokens: keep only alphabetic tokens, and remove stopwords
    tokens = [word.lower() for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)  # Join tokens back into a single string

In [2]:
# Fill NaN values in 'text' column
documents["text"] = documents["text"].fillna("")

# Preprocess documents
documents["processed_text"] = documents["text"].apply(preprocess_ru)

# Convert to list
processed_documents = documents["processed_text"].tolist()
print(processed_documents[:5])  # Check the first 5 processed documents

# Preprocess queries
filtered_queries["processed_query"] = filtered_queries["mt_title"].apply(preprocess_ru)

# Convert to list
processed_queries = filtered_queries["processed_query"].tolist()
print(processed_queries[:5])  # Check the first 5 processed queries


['двое друзей встретились парке гуляя собаками предложил зайти позавтракать ближайшее кафе пустят туда собаками возразил второй первый решительно направился кафе своей немецкой овчаркой хозяин остановил словами сэр нам заходить животными слепой это хозяин извинился проводил собакой столику друг подождал улице пять минут попробовал сделать самое ваш поводырь чихуахуа скептически осведомился хозяин чихуахуа удивился мужчина подсунули анекдот', 'нашли ошибку текст который выделяем смотрим выделили слишком максимальное количество символов попробуйте снова спасибо сообщение отправлено скоро исправим', 'бежит мышка кота прыгает стола попадает бутылку недопитым вином стоящую полу барахтается говорит коту вытащи дай умереть кошмарной смертью убежишь честное слово вытащил первым делом норку шасть сидит кот обижается мышка выходи сказала убежишь мало сказать мужчине женщина нетрезвом виде анекдот', 'председатель федерального ведомства охране конституции германии масен maaßen отправлен отставку о

In [4]:
import pandas as pd
from collections import Counter
import numpy as np

# Preprocessed documents and queries are assumed to be stored in 'processed_documents' and 'processed_queries'
vocabulary = set(word for doc in processed_documents for word in doc.split())

alpha = 0.1  # Smoothing parameter, adjust as necessary

def build_lm(tokens):
    count = Counter(tokens)
    total_tokens = sum(count.values()) or 1  # Avoid division by zero
    # Only store non-zero probabilities
    return {word: (count[word] + alpha) / (total_tokens + alpha * len(vocabulary)) for word in count}

# Rebuild language models using the revised function
doc_models = {idx: build_lm(doc.split()) for idx, doc in enumerate(processed_documents)}

# Same for queries
query_models = {idx: build_lm(query.split()) for idx, query in enumerate(processed_queries)}

# Revised KL divergence calculation using sparse models
def kl_divergence(lm_query, lm_document):
    kl_div = 0
    for word in lm_query:
        prob_q = lm_query[word]
        prob_d = lm_document.get(word, alpha / (alpha * len(vocabulary)))
        kl_div += prob_q * np.log(prob_q / prob_d)
    return kl_div

# Calculate similarities with sparse models
similarities = {q_id: {doc_id: kl_divergence(qm, dm) for doc_id, dm in doc_models.items()} for q_id, qm in query_models.items()}


top_n = 10
similar_documents = {
    q_id: sorted(doc_sims.items(), key=lambda item: item[1])[:top_n] for q_id, doc_sims in similarities.items()
}

# Display results for the first three queries
for q_id in sorted(similar_documents.keys())[:3]:
    print(f"Query {q_id}: Top Documents: {similar_documents[q_id]}")

# Collect results
results = []
for q_id, docs in similar_documents.items():
    for doc_id, score in docs:
        results.append({"query_id": q_id, "doc_id": doc_id, "score": score})

# Display results for the first three queries
for q_id in sorted(similar_documents.keys())[:3]:
    print(f"Query {q_id}: Top Documents: {similar_documents[q_id]}")

Query 0: Top Documents: [(576566, 9.047180309033155e-06), (763003, 9.05379696058879e-06), (912391, 1.100834852327242e-05), (185978, 1.4079516202399173e-05), (215624, 1.4083196592595097e-05), (21735, 1.409335855675659e-05), (208257, 1.5322161251259744e-05), (153980, 1.663792288664647e-05), (548676, 1.9257526735026593e-05), (597102, 1.926050727062691e-05)]
Query 1: Top Documents: [(636472, 1.0947310864330163e-05), (708255, 1.4055652705600609e-05), (643521, 1.928661221551863e-05), (947487, 1.9291859636306558e-05), (905266, 1.9472256113394245e-05), (656037, 2.1551549784895274e-05), (128825, 2.2505460086728497e-05), (773269, 2.353431175313749e-05), (68138, 2.474198244437548e-05), (671288, 2.4759194965592727e-05)]
Query 2: Top Documents: [(743153, -2.8998808025864333e-05), (156209, -2.8666749762192625e-05), (828692, -2.5246256348780563e-05), (188798, -2.1067165071416866e-05), (543739, -1.5320685270809233e-05), (675240, -1.1800896699710048e-05), (91609, -1.1651732184691441e-05), (955415, -1.1

In [8]:
# Collect results in a list
results = []
for q_id, docs in similar_documents.items():
    for doc_id, score in docs:
        results.append({"query_id": q_id, "doc_id": doc_id, "score": score})

# Save results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("query_document_similarity_scores.csv", index=False)

# Load and display the saved CSV to confirm
loaded_df = pd.read_csv("query_document_similarity_scores.csv")
print(loaded_df.head())


   query_id  doc_id     score
0         0  576566  0.000009
1         0  763003  0.000009
2         0  912391  0.000011
3         0  185978  0.000014
4         0  215624  0.000014


# Evaluation

In [10]:
import ir_measures
from ir_measures import nDCG, P, Judged, RBP, AP, RR, R

# Evaluate metrics
evaluation_metrics = ir_measures.calc_aggregate(
    [
        nDCG@20,  # Normalized Discounted Cumulative Gain @20
        P@5,  # Precision @5
        P(rel=1)@5,  # Precision for relevance level >=1 @5
        Judged@10,  # Judged documents @10
        R@100,  # Recall @100
        R@1000,  # Recall @1000
        AP,  # Average Precision
        RR@10,  # Reciprocal Rank @10
    ],
    qrels,  
    results_df  
)

print("Results for LMIR Model Evaluation:")
print(evaluation_metrics)


Results for LMIR Model Evaluation:
{nDCG@20: 0.0085, P@5: 0.008, P(rel=1)@5: 0.006, Judged@10: 0.3, R@100: 0.075, R@1000: 0.091, AP: 0.078, RR@10: 0.08}


Results indicate the performance of the LMIR model in retrieving relevant documents for the queries:

nDCG@20 (0.0085): Very low ranking quality up to the top 20 results, suggesting the model struggles to prioritize highly relevant documents early in the ranking.

P@5 (0.008): Only 0.8% of the top 5 retrieved documents are relevant, indicating poor precision at this rank.

P(rel=1)@5 (0.006): Among documents with relevance ≥1, only 0.6% are in the top 5 results.

Judged@10 (0.3): 30% of the top 10 documents were judged for relevance, implying many unjudged documents in the results.

R@100 (0.075): The model retrieved 7.5% of all relevant documents within the top 100 results.

R@1000 (0.091): Only 9.1% of all relevant documents were retrieved within the top 1000 results.

AP (0.078): The average precision across all recall levels is 7.8%, highlighting consistently low precision.

RR@10 (0.08): The first relevant document appears, on average, near the 12th position (reciprocal rank ~0.08).
